In [1]:
import numpy as np
import sys
import numpy as np
import geopandas as gp
import multiprocessing as mp
import pandas as pd
import libpysal as ps
sys.path.append('/Users/Ziqi/Desktop/mgwr/')
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW
import matplotlib.pyplot as plt
import time
from memory_profiler import memory_usage
%load_ext memory_profiler

/Users/Ziqi/anaconda/lib/python3.5/site-packages/libpysal/io/iohandlers/__init__.py:25: UserWarning: SQLAlchemy and Geomet not installed, database I/O disabled
  warnings.warn('SQLAlchemy and Geomet not installed, database I/O disabled')


In [2]:
#Load Georgia dataset to a pd dataframe
prenz = gp.read_file(ps.examples.get_path('prenzlauer.zip'))

In [3]:
b_y = np.log(prenz['price'].values.reshape((-1, 1)))
b_X = prenz[['review_sco','accommodat','bathrooms']].values 
b_X = (b_X - b_X.mean(axis=0)) / b_X.std(axis=0)
b_y = (b_y - b_y.mean(axis=0)) / b_y.std(axis=0)
u = prenz['X']
v = prenz['Y']
b_coords = np.array(list(zip(u, v)))

In [4]:
%env OMP_NUM_THREADS = 1

env: OMP_NUM_THREADS=1


In [5]:
%%time
gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search()
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit()

192.0
CPU times: user 15.2 s, sys: 1.33 s, total: 16.5 s
Wall time: 9.61 s


In [6]:
%%time
mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search()
print(mgwr_bw)
#mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit()

Iteration 1 SOC: 0.0057381
Bandwidths: 191.0, 1279.0, 82.0, 2200.0
Iteration 2 SOC: 0.0013604
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
Iteration 3 SOC: 0.0002223
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
Iteration 4 SOC: 2.48e-05
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
Iteration 5 SOC: 3.1e-06
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
[ 191. 1279.   79. 2200.]
CPU times: user 1min 17s, sys: 1.18 s, total: 1min 18s
Wall time: 1min 12s


In [7]:
n_proc = 2
pool = mp.Pool(n_proc)

In [8]:
%%time
gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search(pool=pool)
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit(pool=pool)

192.0
CPU times: user 342 ms, sys: 49.9 ms, total: 391 ms
Wall time: 4.89 s


In [9]:
%%time
mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search(pool=pool)
print(mgwr_bw)
#mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit()

Iteration 1 SOC: 0.0057381
Bandwidths: 191.0, 1279.0, 82.0, 2200.0
Iteration 2 SOC: 0.0013604
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
Iteration 3 SOC: 0.0002223
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
Iteration 4 SOC: 2.48e-05
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
Iteration 5 SOC: 3.1e-06
Bandwidths: 191.0, 1279.0, 79.0, 2200.0
[ 191. 1279.   79. 2200.]
CPU times: user 4.99 s, sys: 424 ms, total: 5.42 s
Wall time: 48.9 s
